# Payoff Matrices (part 4)

> This module contains payoff matrices for different evolutionary games
>
> Part 4 contains payoff matrices for the following games
> - A public threat level model of AI races

In [1]:
# | default_exp `dummy_exp_2`


In [2]:
# | hide
# We could like to export all payoff matrices to the same module, `payoffs`.
# However,  we must set a default_exp and if we use the same module as
# another notebook, we will overwrite all existing exports with those in
# this file. Hence we set a dummy default_exp, `dummy_exp`, and each time
# we export a cell, we specify the `payoffs` module.


In [3]:
# | hide
# | export
from nbdev.showdoc import *
from fastcore.test import test_eq, test_close
import collections
import functools
from gh_pages_example.utils import *
from gh_pages_example.types import *
from gh_pages_example.methods import *
from gh_pages_example.model_utils import *
from gh_pages_example.payoffs import *
import copy
import itertools
import math
import typing

import fastcore.test
import more_itertools
import numpy as np
import nptyping


/home/ethos/git/gh-pages-example/gh_pages_example/model_utils.py:299: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (ind not in allowed_inds) and (str(ind) not in allowed_inds):
/home/ethos/git/gh-pages-example/gh_pages_example/methods.py:260: ComplexWarning: Casting complex values to real discards the imaginary part
  ergodic = np.array(V.transpose(0, 2, 1)[y], dtype=float)


NameError: name 'build_payoffs' is not defined

In [ ]:
np.set_printoptions(suppress=True)  # don't use scientific notation

In [ ]:
# | export payoffs
@method(build_payoffs, "dsair_with_punishment_v1")
def build_payoffs(models):
    """The payoffs of a DSAIR race where punishments are levied."""
    names1 = ['b', 'c', 's', 'p', 'B', 'W']
    names2 = ['pfo_l', 'pfo_h', 'λ', 'r_l', 'r_h', 'g']
    b, c, s, p, B, W = [models[k] for k in names1]
    pfo_l, pfo_h, λ, r_l, r_h, g = [models[k] for k in names2]
    collective_risk = models.get('collective_risk', 0)
    risk_shared = (1 - (1-p)*collective_risk)
    mix = models.get('incentive_mix', 0)
    
    k = models.get('decisiveness', 100)
    # Speed impact of regulators when they catch 1 or 2 safety violators
    phi_h = models.get('phi_h', 1/s)
    phi2_h = models.get('phi2_h,', 1/s)
    phi_l = models.get('phi_l', 1/s)
    phi2_l = models.get('phi2_l', 1/s)
    # Tullock contest to determine which firm wins after
    # one safety violator is caught
    caught_loses_h = ((s * phi_h)**k + 1)**(-1)
    caught_loses_l = ((s * phi_l)**k + 1)**(-1)
    # Tullock contest to determine whether any firm wins if they are both
    # safety violators who were caught by the regulator
    both_caught_fail_h = ((s * phi2_h)**k + 1)**(-1)
    both_caught_fail_l = ((s * phi2_l)**k + 1)**(-1)
    
    Π_h11 = B / (2*W) + b/2 - c
    Π_h12 = ((1 - pfo_h) * b / (s+1) * risk_shared
             + pfo_h * caught_loses_h * (b + B / W)
             - c)
    Π_h21 = (p * (1 - pfo_h) * (s*b / (s + 1) + s * B / W)
             + (pfo_h * (1 - caught_loses_h)
                * B / W))
    Π_h22 = (p * (1 - pfo_h**2) * (b/2 + s*B/(2*W)) * risk_shared
             + (pfo_h**2 * (1 - both_caught_fail_h)
                * B/(2*W)))
    
    payoffs = {}
    payoffs['1-1'] = {'P1': Π_h11,
                      'P2': Π_h11}
    payoffs['1-2'] =  {'P1': Π_h21,
                       'P2': Π_h12}
    payoffs['2-1'] =  {'P1': Π_h12,
                       'P2': Π_h21}
    payoffs['2-2'] =  {'P1': Π_h22,
                       'P2': Π_h22}
    
    return {**models, "payoffs": payoffs}


In [ ]:
# | export payoffs
@method(build_payoffs, "public_threat_level_v1")
def build_payoffs(models):
    """The payoffs of a DSAIR race where punishments are levied when
    the public threat level is sufficiently high. The public threat
    level depends on the state of the population."""
    models1 = copy.deepcopy(models)
    models2 = copy.deepcopy(models)
    models1['phi_h'] = models.get('phi_h', 1/models["s"])
    models2['phi_h'] = models.get('phi_h', 1)
    payoffs1 = thread_macro({**models1,
                             "payoffs-key": "dsair_with_punishment_v1",},
                            build_payoffs,
                            (get, "payoffs"))
    payoffs2 = thread_macro({**models2,
                             "payoffs-key": "dsair_with_punishment_v1",},
                            build_payoffs,
                            (get, "payoffs"))
    payoffs = {"severity-1": payoffs1, "severity-2": payoffs2}
    return {**models, "payoffs": payoffs}

@multi
def evaluate_payoffs(models):
    return models.get("payoffs_key")

@method(evaluate_payoffs, "public_threat_level_v1")
def evaluate_payoffs(models):
    names1 = ["profile", "player", "payoffs", "population_state"]
    names2 = ["Z", "severity"]
    profile, player, payoffs, population_state = [models[k] for k in names1]
    Z, severity = [models[k] for k in names2]
    strategy_counts = population_state['strategy_counts']
    unsafe_frequency = strategy_counts.get("1", 0)
    if unsafe_frequency > severity * Z["S1"]:
        payoffs = payoffs["severity-1"]
    else:
        payoffs = payoffs["severity-2"]
    return payoffs[profile][player]

In [7]:
# | hide
import nbdev
nbdev.nbdev_export()
